In [62]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# 프로젝트 경로 가져오기(현재경로의 상위상위폴더)
project_path = os.path.dirname(os.path.dirname(os.getcwd()))
ibmhr_path = os.path.join(project_path, 'data', 'interim','ibmhr.xlsx')
ibmhr = pd.read_excel(ibmhr_path)
ibmhr.columns
ibmhr

,연령,퇴직여부,본부,출퇴근거리,교육,전공,사원번호,성별,직무,결혼여부,월급,이직횟수,OT여부,성과평가,총경력년수,근속년수,부서,d_월급
0,41,예,영업,1,2,생명 과학,1,여성,영업 간부,미혼,5993,8,예,3,8,6,영업,599.3
1,49,아니오,연구개발,8,1,생명 과학,2,남성,연구 과학자,기혼,5130,1,아니오,4,10,10,의료연구팀,513.0
2,37,예,연구개발,2,2,기타,4,남성,실험실 기술자,미혼,2090,6,예,3,7,0,의료제조팀,209.0
3,33,아니오,연구개발,3,4,생명 과학,5,여성,연구 과학자,기혼,2909,1,예,3,8,8,의료연구팀,290.9
4,27,아니오,연구개발,2,1,의료,7,남성,실험실 기술자,기혼,3468,9,아니오,3,6,2,의료제조팀,346.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,아니오,연구개발,23,2,의료,2061,남성,실험실 기술자,기혼,2571,4,아니오,3,17,5,의료제조팀,257.1
1466,39,아니오,연구개발,6,1,의료,2062,남성,의료 대표,기혼,9991,4,아니오,3,9,7,연구개발,999.1
1467,27,아니오,연구개발,4,3,생명 과학,2064,남성,제조 책임자,기혼,6142,1,예,4,6,6,의료제조팀,614.2
1468,49,아니오,영업,2,3,의료,2065,남성,영업 간부,기혼,5390,2,아니오,3,17,9,영업,539.0


In [63]:
ibmhr.columns

Index(['연령', '퇴직여부', '본부', '출퇴근거리', '교육', '전공', '사원번호', '성별', '직무', '결혼여부',
       '월급', '이직횟수', 'OT여부', '성과평가', '총경력년수', '근속년수', '부서', 'd_월급'],
      dtype='object')

In [64]:
# 필요한 데이터만 추출
ibmhr =ibmhr[['연령', '퇴직여부', '본부', '출퇴근거리', '교육', '전공', '사원번호',
              '성별', '직무', '결혼여부', '월급', '이직횟수', 'OT여부', '성과평가',
              '총경력년수', '근속년수', '부서']]

In [65]:
# 한국 급여단위로 환산
ibmhr = ibmhr.reset_index(drop=True) # 우선 인덱스를 초기화해야함
ibmhr['d_월급'] = ibmhr['월급'] / 10
ibmhr

,연령,퇴직여부,본부,출퇴근거리,교육,전공,사원번호,성별,직무,결혼여부,월급,이직횟수,OT여부,성과평가,총경력년수,근속년수,부서,d_월급
0,41,예,영업,1,2,생명 과학,1,여성,영업 간부,미혼,5993,8,예,3,8,6,영업,599.3
1,49,아니오,연구개발,8,1,생명 과학,2,남성,연구 과학자,기혼,5130,1,아니오,4,10,10,의료연구팀,513.0
2,37,예,연구개발,2,2,기타,4,남성,실험실 기술자,미혼,2090,6,예,3,7,0,의료제조팀,209.0
3,33,아니오,연구개발,3,4,생명 과학,5,여성,연구 과학자,기혼,2909,1,예,3,8,8,의료연구팀,290.9
4,27,아니오,연구개발,2,1,의료,7,남성,실험실 기술자,기혼,3468,9,아니오,3,6,2,의료제조팀,346.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,아니오,연구개발,23,2,의료,2061,남성,실험실 기술자,기혼,2571,4,아니오,3,17,5,의료제조팀,257.1
1466,39,아니오,연구개발,6,1,의료,2062,남성,의료 대표,기혼,9991,4,아니오,3,9,7,연구개발,999.1
1467,27,아니오,연구개발,4,3,생명 과학,2064,남성,제조 책임자,기혼,6142,1,예,4,6,6,의료제조팀,614.2
1468,49,아니오,영업,2,3,의료,2065,남성,영업 간부,기혼,5390,2,아니오,3,17,9,영업,539.0


In [61]:
# 데이터를 엑셀파일로 변환
ibmhr.to_excel(project_path + "/data/processed/ibmhr.xlsx", index=False)
